# Melanoma recognizer

## Initial configuration

### Import packages

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import Image
import scipy

### Set paths to datasets

In [ ]:
train_data_dir = 'data/train'
validation_data_dir = 'data/test'

### Initialize parameters

In [ ]:
img_width, img_height = 224, 224
batch_size = 32
epochs = 50

## Preprocessing

### Data augmentation and normalization

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)

### Create iterators for data loading

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


## Prepare CNN model

### Create model

In [ ]:
model = Sequential()

### Add convolutional layers

In [ ]:
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(img_width, img_height, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

### Connect layers

In [ ]:
# Flattening
model.add(Flatten())
model.add(Dense(512, activation='relu'))

# Dropout to avoid overfitting
model.add(Dropout(0.5))  

### Output layer

In [ ]:
model.add(Dense(1, activation='sigmoid'))

### Print model summary

In [ ]:
model.summary()

### Training

In [ ]:
optimizer = Adam(learning_rate = 0.001)

model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)

### Save the model

In [ ]:
model.save('melanoma_classifier_model.h5')

## Evaluation

### Evaluate the model

In [ ]:
results = model.evaluate(validation_generator)
print(f"Validation Loss: {results[0]}")
print(f"Validation Accuracy: {results[1]}")